In [1]:
import joblib
import tensorflow as tf
import pandas as pd
from keras.losses import mean_squared_error

custom_objects = {'mse': mean_squared_error}
model = tf.keras.models.load_model('models/feature_extractor_model.keras', custom_objects=custom_objects)
rfnew = joblib.load('models/rf_model1.pkl')
rf_model = joblib.load('models/rf_model.pkl')
scaler = joblib.load('models/scaler.pkl')
knn = joblib.load('models/knn.pkl')

In [2]:
#preprocess and predict using rf
def preprocess_message(can_message):
    
    def convert_to_int(x):
        if isinstance(x, str):
            try:
                return int(x, 16)
            except ValueError:
                return x
        return x
    
    for i in range(8):
        can_message[f'data[{i}]'] = convert_to_int(can_message[f'data[{i}]'])
    
    can_message['can_id'] = convert_to_int(can_message['can_id'])

    can_message = pd.DataFrame([can_message])
    
    can_message = can_message.drop(columns=['ignore'])
    can_message = can_message.drop(columns=['dlc'])
    preprocessed_message = scaler.transform(can_message)
    
    return preprocessed_message

def predict_can_message(can_message):
    preprocessed_message = preprocess_message(can_message)
    
    features = model.predict(preprocessed_message)
    
    prediction = rf_model.predict(features)
    
    return prediction[0]


In [3]:
# Example CAN message - predicted using rf
can_message = {
    'time': 1673043786.214782,
    'can_id': 0x3D1,
    'ignore': 0x000,
    'dlc': 8,
    'data[0]': 'FF',
    'data[1]': '1F',
    'data[2]': '00',
    'data[3]': '00',
    'data[4]': 'FD',
    'data[5]': '00',
    'data[6]': '00',
    'data[7]': '00'
}
#1673043786.214782,3D1,FF1F0000FD000000
# Predict the message
prediction = predict_can_message(can_message)
print(f'Anomolous:{bool(prediction)}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Anomolous:True


In [5]:
a1 = {
    'time': 1672281227.925416,
    'can_id': 0x1F1,
    'ignore': 0x000,
    'dlc': 8,
    'data[0]': 'AE',
    'data[1]': '0E',
    'data[2]': '00',
    'data[3]': '00',
    'data[4]': '08',
    'data[5]': '00',
    'data[6]': '00',
    'data[7]': '7A'
}
#0.010908,0153,000,8,00,80,10,ff,00,ff,50,de
# Predict the message
print(f'Anomolous:{bool(predict_can_message(a1))}' )    #rf
print("\nnext\n")
print(f'Anomolous:{bool(predict_can_message2(a1))}' )   #knn

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Anomolous:True

next

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Anomolous:True


In [6]:
#string format
from sklearn.preprocessing import StandardScaler
def preprocess_message(can_message_str):
    parts = can_message_str.split(',')
    timestamp = float(parts[0])
    can_id = parts[1]
    data = parts[2]
    data_bytes = [data[i:i+2] for i in range(0, len(data), 2)]
    while len(data_bytes) < 8:
        data_bytes.append('00')
    def convert_to_int(x):
        if isinstance(x, str):
            try:
                return int(x, 16)
            except ValueError:
                return x
        return x
    for i in range(8):
        data_bytes[i] = convert_to_int(data_bytes[i])
    can_id = convert_to_int(can_id)
    can_message = {
        'time': timestamp,
        'can_id': can_id,
        'data[0]': data_bytes[0],
        'data[1]': data_bytes[1],
        'data[2]': data_bytes[2],
        'data[3]': data_bytes[3],
        'data[4]': data_bytes[4],
        'data[5]': data_bytes[5],
        'data[6]': data_bytes[6],
        'data[7]': data_bytes[7]
    }
    can_message_df = pd.DataFrame([can_message])
    preprocessed_message = scaler.transform(can_message_df)
    return preprocessed_message
def predict_can_message1(can_message_str):
    preprocessed_message = preprocess_message(can_message_str)
    features = model.predict(preprocessed_message)
    prediction = knn.predict(features)
    return prediction[0]
# CAN message in string format
can_message_str = "1672081958.389626,0F9,0000400000000016"
prediction = predict_can_message1(can_message_str)
print(f'Anomalous: {bool(prediction)}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Anomalous: True


In [5]:
#process csv
import pandas as pd
from sklearn.preprocessing import StandardScaler

def preprocess_messageNEW(can_message_str, scaler):
    parts = can_message_str.split(',')
    timestamp = float(parts[0])
    can_id = parts[1]
    data = parts[2]

    data_bytes = [data[i:i+2] for i in range(0, len(data), 2)]
    while len(data_bytes) < 8:
        data_bytes.append('00')

    def convert_to_int(x):
        if isinstance(x, str):
            try:
                return int(x, 16)
            except ValueError:
                return x
        return x
    
    for i in range(8):
        data_bytes[i] = convert_to_int(data_bytes[i])
    
    can_id = convert_to_int(can_id)

    can_message = {
        'time': timestamp,
        'can_id': can_id,
        'data[0]': data_bytes[0],
        'data[1]': data_bytes[1],
        'data[2]': data_bytes[2],
        'data[3]': data_bytes[3],
        'data[4]': data_bytes[4],
        'data[5]': data_bytes[5],
        'data[6]': data_bytes[6],
        'data[7]': data_bytes[7]
    }

    can_message_df = pd.DataFrame([can_message])
    
    preprocessed_message = scaler.transform(can_message_df)
    
    return preprocessed_message

def predict_can_messagesNEW(file_path, model, knn, scaler):
    with open(file_path, 'r') as file:
        can_messages = file.readlines()
    
    predictions = []
    
    for can_message_str in can_messages:
        preprocessed_message = preprocess_messageNEW(can_message_str.strip(), scaler)
        
        features = model.predict(preprocessed_message)
        
        prediction = knn.predict(features)
        predictions.append(bool(prediction[0]))
    
    return predictions

file_path = "data.txt"

predictions = predict_can_messagesNEW(file_path, model, knn, scaler)

for i, prediction in enumerate(predictions):
    print(f'CAN message {i+1} - Anomalous: {prediction}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
CAN message 1 - Anomalous: False
CAN message 2 - Anomalous: False
CAN message 3 - Anomalous: False
CAN message 4 - Anomalous: True
CAN message 5 - Anomalous: True
CAN message 6 - Anomalous: True
CAN message 7 - Anomalous: False
CAN message 8 - Anomalous: True
